# Imports and Data Loading

In [41]:
# the notebook's main objective is to filter and prepare the dataset to train a summarizer on it.
import os, sys
from pathlib import Path
HOME = os.getcwd()

current = HOME 
while 'src' not in os.listdir(current):
    current = Path(current).parent

PARENT_DIR = str(current)
DATA_FOLDER = os.path.join(PARENT_DIR, 'src', 'data')
data_path = os.path.join(DATA_FOLDER, 'filtered.tsv')
sys.path.append(str(current))
sys.path.append(os.path.join(str(current), 'data_analysis'))
sys.path.append(os.path.join(str(current), 'evaluation'))
sys.path.append(os.path.join(str(current), 'text_processing'))
sys.path.append(os.path.join(str(current), 'models'))

In [42]:
from datasets import load_dataset
all_data = load_dataset('csv', data_files=os.path.join(DATA_FOLDER, 'all_data_processed.csv'), split='train')
all_data = all_data.filter(lambda s: (isinstance(s['source'], str) and isinstance(s['target'], str)))

In [43]:
all_data

Dataset({
    features: ['source', 'target'],
    num_rows: 597519
})

In [44]:
from src.text_processing import preprocess as pr
# the next step is to filter the dataset
def filter_data(sample):
    """This function receives  a batch of samples from the original data and filters those whose 'source' text is shorter than the 'target' text."""
    # first tokenize each 'source' and 'target' fields
    source = pr.tokenize(sample['source'], tokenizer_type='word')
    target = pr.tokenize(sample['target'], tokenizer_type='word')
    return len(source) > len(target)

summary_data = all_data.filter(filter_data)
# save the data
summary_data.to_csv(os.path.join(DATA_FOLDER, 'summarized_data.csv'), index=False)

Creating CSV from Arrow format:   0%|          | 0/277 [00:00<?, ?ba/s]

Creating CSV from Arrow format: 100%|██████████| 277/277 [00:01<00:00, 239.18ba/s]


31406141

In [45]:
import src.data_preparation.prepare_data as pdr
sample = summary_data.select(range(5000))
train_data, val_data, test_data = pdr.data_split(all_data=sample)

# DataLoaders

In [46]:
# the next step is to tokenize the data  
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
CHECKPOINT = 't5-small'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TOKENIZER = AutoTokenizer.from_pretrained(CHECKPOINT)
MODEL = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT).to(DEVICE)

In [47]:
TASK_PREFIX = 'summarize: '

def prepare_labeled_data(batch):
    # add the task predix to each sentence
    inputs = [TASK_PREFIX + doc for doc in batch["source"]]
    # tokenize 'x'
    model_inputs = TOKENIZER(inputs, truncation=True, max_length=1028)
    # tokenize 'y'  
    labels = TOKENIZER(text_target=batch["target"], truncation=True)
    # add it to the model's input
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [48]:
train_data = train_data.map(prepare_labeled_data, batched=True).remove_columns(['source', 'target'])
val_data = val_data.map(prepare_labeled_data, batched=True).remove_columns(['source', 'target'])

In [49]:
# create a DataCollator for padding for the sequence to sequence models
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer=TOKENIZER, model=CHECKPOINT)
# we are now ready to create the dataloader
from torch.utils.data import DataLoader
train_dl = DataLoader(dataset=train_data, batch_size=4, shuffle=True, collate_fn=data_collator)
val_dl = DataLoader(dataset=val_data, batch_size=4, shuffle=False, collate_fn=data_collator)

In [50]:
# make sure the data is loaded correctly
b1, b2 = next(iter(train_dl)), next(iter(val_dl))

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


# Model Training

## Training Code

## Train utilities

In [51]:
from src.evaluation import toxicity_classication as tc
singleton_obj = tc.EvalutionSingletonInitializer()
tx_classifier, tx_tokenizer, tx_device = singleton_obj.get_toxic_classifier(), singleton_obj.get_toxic_tokenizer(), singleton_obj.get_device()
# let's define some of the training parameters
from torch.optim import Adam
from torch.optim.lr_scheduler import LinearLR

# lr: the same as the one used in the/home/ayhem18/DEV/My_Kaggle_Repo/pytorch_modular/pytorch_utilities.py
optimizer = Adam(MODEL.parameters(), lr=2 * 10 ** -5)
scheduler = LinearLR(optimizer=optimizer, start_factor=1, end_factor=0.5,total_iters=100)

In [52]:
# let's write a function to compute the summarization + toxicity loss
from src.evaluation.toxicity_classication import EvalutionSingletonInitializer
from torch.nn.functional import softmax
from typing import Union

def toxic_summary_model_loss(output_decoded: torch.Tensor, 
                             device,
                             return_tensor: bool=False) -> Union[float, torch.Tensor]:
    
    singleton_obj = EvalutionSingletonInitializer()
    tc_tokenizer, tc_classifier = singleton_obj.get_toxic_tokenizer(), singleton_obj.get_toxic_classifier()

    # make sure to freeze their parameters
    for p in tc_classifier.parameters():
        p.requires_grad = False

    tc_classifier.to(device)
    # tokenize
    model_input = tc_tokenizer(output_decoded, return_tensors='pt', padding=True, truncation=True)
    # set the input to the device
    model_input = {k: v.to(device) for k, v in model_input.items()}
    # pass through the model
    output = tc_classifier(**model_input)
    
    loss = torch.mean(softmax(output.logits, dim=1)[:, 1])
    
    if return_tensor: 
        loss.requires_grad=True
        return loss
    
    return loss.item()


In [53]:
from src.models.summarizer import summarizer  as ss
import src.training_utilities.exp_tracking as et
import importlib 
importlib.reload(ss)
importlib.reload(et)

_, _, best_model = ss.train_custom_summarizer(train_dataloader=train_dl, 
                                            val_dataloader=val_dl,
                                            summary_model=MODEL,
                                            summary_tokenizer=TOKENIZER, 
                                            toxicity_loss_function=toxic_summary_model_loss,
                                            toxicity_coeff=0.5, 
                                            optimizer=optimizer, 
                                            scheduler=scheduler, 
                                            num_epochs=20,   
                                            report_per_epoch=1,
                                            log_dir=os.path.join(HOME, 'runs')
                                            )


[INFO] Created SummaryWriter, saving to: /home/ayhem18/DEV/TextDetoxification/src/data_analysis/runs/experience_3...


  0%|          | 0/20 [00:00<?, ?it/s]

/home/ayhem18/DEV/TextDetoxification/env/lib/python3.11/site-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
  5%|▌         | 1/20 [00:51<16:18, 51.50s/it]

Training losses
train_loss: 2.058954809506734
val_loss: 1.811967327594757
val_toxic_loss: 0.4798521736264229
##################################################



 10%|█         | 2/20 [01:44<15:43, 52.41s/it]

...

Training losses
train_loss: 1.3683720229069392
val_loss: 1.6363922333717347
val_toxic_loss: 0.2932124473992735
##################################################



 95%|█████████▌| 19/20 [16:42<00:52, 52.18s/it]

Training losses
train_loss: 1.3503433749824763
val_loss: 1.633616714477539
val_toxic_loss: 0.30257517254445704
##################################################



100%|██████████| 20/20 [17:33<00:00, 52.69s/it]

Training losses
train_loss: 1.3333475915218393
val_loss: 1.6379020452499389
val_toxic_loss: 0.3058235085057095
##################################################



In [54]:
for i in range(0, len(val_data), 20):
    input_ids = val_data[i]['input_ids']
    attention_mask = val_data[i]['attention_mask']
    labels = val_data[i]['labels']

    print(f"source: {TOKENIZER.decode(input_ids, skip_special_tokens=True)}")
    print(f"target: {TOKENIZER.decode(labels, skip_special_tokens=True)}")

    outputs = best_model.generate(
        input_ids=torch.tensor(input_ids).unsqueeze(0).to('cuda'),
        attention_mask=torch.tensor(attention_mask).unsqueeze(0).to('cuda'),
        max_length=512,
        num_beams=5,
        early_stopping=True
    )

    print(f"generated :{TOKENIZER.decode(outputs[0], skip_special_tokens=True)}")
    print("#" * 100)


source: summarize: we don't fear these demons. we destroy them.
target: we don't fight demons, we destroy them.
generated :we don't fear these demons.
####################################################################################################
source: summarize: the old witch shook her head.
target: she shook her head.
generated :the old witch shook her head.
####################################################################################################
source: summarize: you want me to fucking leave her
target: you want to leave it
generated :you want me to leave her
####################################################################################################
source: summarize: why don't you get rid of her before you go, huh
target: why not make amends before you leave, huh
generated :why don't you get rid of her before you go
####################################################################################################
source: summarize: darn it. fractions a